## Fire Detection TinyML Model


In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
import tensorflow as tf

import tensorflow.keras as keras
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, GlobalMaxPooling2D, BatchNormalization, Activation

from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [3]:
model = keras.Sequential()


model.add(Conv2D(12, kernel_size=3, strides=2,input_shape=(160,120,3), kernel_constraint=tf.keras.constraints.MaxNorm(1),activation='relu', padding='same'))

model.add(MaxPooling2D(pool_size=2, strides=2, padding='same'))

model.add(Conv2D(36, kernel_size=3, strides=2,activation='relu', padding='same'))

model.add(Dropout(0.2))

model.add(Conv2D(72, kernel_size=3, strides=2, activation='relu', padding='same'))

model.add(Dropout(0.2))
model.add(GlobalMaxPooling2D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))


In [4]:
opt = keras.optimizers.Adam()
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])


# Data Preprocessing


In [5]:
#collected dataset of around 1200 photos

train_datagen = ImageDataGenerator(
        rescale=1./255,
        horizontal_flip=True, vertical_flip =True,brightness_range=[0.8,1.2],zoom_range=[0.8,1.2])
test_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = test_datagen.flow_from_directory(
        '/content/drive/MyDrive/FireDataSplit/val',
        target_size=(160, 120),
        color_mode='rgb',
        batch_size=64,
         class_mode='categorical')
train_generator = train_datagen.flow_from_directory(
        '/content/drive/MyDrive/FireDataSplit/train',
        target_size=(160, 120),
        batch_size=64,
        color_mode='rgb',
        class_mode='categorical')

Found 3000 images belonging to 2 classes.
Found 9008 images belonging to 2 classes.


# Training


In [6]:
history = model.fit(
        train_generator,
        epochs=100,
        validation_data=validation_generator,
      verbose=1)

Epoch 1/100
141/141 [==============================] - 3374s 24s/step - loss: 0.4511 - accuracy: 0.7931 - val_loss: 0.3246 - val_accuracy: 0.8823
Epoch 2/100
141/141 [==============================] - 70s 499ms/step - loss: 0.3196 - accuracy: 0.8763 - val_loss: 0.2670 - val_accuracy: 0.9070
Epoch 3/100
141/141 [==============================] - 70s 497ms/step - loss: 0.2878 - accuracy: 0.8897 - val_loss: 0.2441 - val_accuracy: 0.9250
Epoch 4/100
141/141 [==============================] - 69s 487ms/step - loss: 0.2581 - accuracy: 0.9062 - val_loss: 0.2145 - val_accuracy: 0.9270
Epoch 5/100
141/141 [==============================] - 69s 488ms/step - loss: 0.2327 - accuracy: 0.9177 - val_loss: 0.2369 - val_accuracy: 0.9067
Epoch 6/100
141/141 [==============================] - 68s 483ms/step - loss: 0.2313 - accuracy: 0.9162 - val_loss: 0.2004 - val_accuracy: 0.9380
Epoch 7/100
141/141 [==============================] - 68s 481ms/step - loss: 0.2222 - accuracy: 0.9187 - val_loss: 0.2051 -

In [7]:
#save the model to use later
model.save("/content/drive/MyDrive/firedetect") 


INFO:tensorflow:Assets written to: /content/drive/MyDrive/firedetect/assets


# Quantizing The Model


In [8]:
import glob
import os
import pandas as pd
import numpy as np

In [9]:
converter = tf.lite.TFLiteConverter.from_saved_model("/content/drive/MyDrive/firedetect")
model_no_quant_tflite = converter.convert()

# Save the model to disk
open("float.tflite", "wb").write(model_no_quant_tflite)
def representative_dataset():
  for filename in glob.glob("/content/drive/MyDrive/FireDataSplit/val" + "/*/*.jpg"):
    img = keras.preprocessing.image.load_img(filename, target_size=(160, 120))
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create batch axis for images, labels in train_ds.take(1):
    yield([img_array])
# Set the optimization flag.
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# Enforce integer only quantization
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
# Provide a representative dataset to ensure we quantize correctly.
converter.representative_dataset = representative_dataset
model_tflite = converter.convert()

# Save the model to disk
open("quantized.tflite", "wb").write(model_tflite)

42336

##Comparing Sizes Between Models


In [ ]:
def get_dir_size(dir):
  size = 0
  for f in os.scandir(dir):
    if f.is_file():
      size += f.stat().st_size
    elif f.is_dir():
      size += get_dir_size(f.path)
  return size

# Calculate size
size_tf = get_dir_size("/content/drive/MyDrive/firedetect")
size_no_quant_tflite = os.path.getsize("float.tflite")
size_tflite = os.path.getsize("quantized.tflite")

# Compare size
pd.DataFrame.from_records(
    [["TensorFlow", f"{size_tf} bytes", ""],
     ["TensorFlow Lite", f"{size_no_quant_tflite} bytes ", f"(reduced by {size_tf - size_no_quant_tflite} bytes)"],
     ["TensorFlow Lite Quantized", f"{size_tflite} bytes", f"(reduced by {size_no_quant_tflite - size_tflite} bytes)"]],
     columns = ["Model", "Size", ""], index="Model")

##Testing It Out

In [ ]:
def predict_tflite(tflite_model, filename):
  img = keras.preprocessing.image.load_img(filename, target_size=(160, 120))
  img_array = keras.preprocessing.image.img_to_array(img)
  img_array = tf.expand_dims(img_array, 0)

  # Initialize the TFLite interpreter
  interpreter = tf.lite.Interpreter(model_content=tflite_model)
  interpreter.allocate_tensors()

  input_details = interpreter.get_input_details()[0]
  output_details = interpreter.get_output_details()[0]

  # If required, quantize the input layer (from float to integer)
  input_scale, input_zero_point = input_details["quantization"]
  if (input_scale, input_zero_point) != (0.0, 0):
    img_array = np.multiply(img_array, 1.0 / input_scale) + input_zero_point
    img_array = img_array.astype(input_details["dtype"])
  
  # Invoke the interpreter
  interpreter.set_tensor(input_details["index"], img_array)
  interpreter.invoke()
  pred = interpreter.get_tensor(output_details["index"])[0]
  
  # If required, dequantized the output layer (from integer to float)
  output_scale, output_zero_point = output_details["quantization"]
  if (output_scale, output_zero_point) != (0.0, 0):
    pred = pred.astype(np.float32)
    pred = np.multiply((pred - output_zero_point), output_scale)
  
  predicted_label_index = np.argmax(pred)
  predicted_score = pred[predicted_label_index]
  return (predicted_label_index, predicted_score)

In [ ]:
print(predict_tflite(model_tflite,"fire1.jpg")) #fires are (0,1) and non fires are a (1,1)

##Exporting the Quantized Model

In [ ]:
!apt-get update && apt-get -qq install xxd

In [ ]:
!xxd -i quantized.tflite > model_data.cc